In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
Scipy not supported!


In [2]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

110 documents
36 classes ['addiction', 'affirmation_distraction_addiction', 'anxiety', 'challenge_thought_anxiety', 'depression', 'explain_distraction_addiction', 'explainreccdist_depression', 'first_substance_addiction', 'go_plan_for_things', 'goodbye', 'greeting', 'how_wreck', 'ideas_wrecking', 'need_help', 'need_help_adiction', 'need_ideas_addiction', 'no_distraction_addiction', 'no_distraction_addiction_after', 'plan_for_thing', 'ques_to_q1_anxiety', 'question1_anxiety', 'question1_depression', 'question_to_question1_depression', 'question_to_question2_anxiety', 'question_to_question2_depression', 'question_to_question3_depression', 'questtion_to_question4_depression', 'stating_problem', 'thanks', 'the_end', 'the_end_bad', 'things_addiction', 'what_to_do_addiction2', 'work_on_it', 'work_on_it_anxiety', 'wrote_everything_addiction']
111 unique stemmed words ["'s", ',', '.', '..', '...', '1', '3', 'a', 'addict', 'again', 'al', 'anxy', 'any', 'anyon', 'ar', 'assum', 'bad', 'be', 'bet'

In [4]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\samit\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [5]:
# reset underlying graph data
#tf.reset_default_graph()
tf.compat.v1.reset_default_graph()

# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)

net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=2000, batch_size=16, show_metric=True)
model.save('model.tflearn')

Training Step: 13999  | total loss: 0.37206 | time: 0.018s
| Adam | epoch: 2000 | loss: 0.37206 - acc: 0.7908 -- iter: 096/110
Training Step: 14000  | total loss: 0.35516 | time: 0.022s
| Adam | epoch: 2000 | loss: 0.35516 - acc: 0.7974 -- iter: 110/110
--
INFO:tensorflow:C:\Users\samit\MACHINE LEARNING\chatbot BE project\New\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [7]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
['addiction', 'affirmation_distraction_addiction', 'anxiety', 'challenge_thought_anxiety', 'depression', 'explain_distraction_addiction', 'explainreccdist_depression', 'first_substance_addiction', 'go_plan_for_things', 'goodbye', 'greeting', 'how_wreck', 'ideas_wrecking', 'need_help', 'need_help_adiction', 'need_ideas_addiction', 'no_distraction_addiction', 'no_distraction_addiction_after', 'plan_for_thing', 'ques_to_q1_anxiety', 'question1_anxiety', 'question1_depression', 'question_to_question1_depression', 'question_to_question2_anxiety', 'question_to_question2_depression', 'question_to_question3_depression', 'questtion_to_question4_depression', 'stating_problem', 'thanks', 'the_end', 'the_end_bad', 'things_addiction', 'what_to_do_addiction2', 'work_on_it', 'wo

In [8]:
print(model.predict([p]))


[[9.6188188e-08 1.6355199e-03 2.8713887e-07 1.6551047e-10 3.1486913e-08
  4.2345919e-04 7.8916270e-01 5.8737991e-05 4.0042214e-06 5.4795458e-04
  6.4495414e-02 3.3598771e-10 1.6540794e-02 3.0242645e-11 5.1592215e-09
  1.0529591e-04 6.9527273e-11 6.2945663e-11 3.7423099e-08 2.3454027e-03
  1.3842462e-09 1.7590803e-09 3.1554676e-03 2.2672853e-03 1.0999148e-01
  7.9787476e-03 1.1296995e-03 1.9303061e-10 6.2488361e-05 4.7614861e-07
  2.3536145e-08 2.7689202e-06 9.1451970e-05 1.4576324e-07 1.6727718e-07
  5.6143890e-09]]


In [9]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )